### Import required Packages ###
Numpy, and pandas are required to be installed

In [1]:
import math
import numpy as np
import pandas as pd
# Allow chained assignments without a warn message
pd.options.mode.chained_assignment = None  # default='warn'
from statistics import mean
from random import sample

### Initialization ###

In [2]:
# Problem Initialization.R (Initialization.R)
n_size = 200 # Problem size
theta_deg = 0 # Initial degradation
delta = 1 # maximal degradation
cost_f = 100 # maximum predictive maintenance cost
cost_zero = 1000 # minimum predictive maintenance cost spent when the machine reaches a full degradation ??
p_vec = np.random.uniform(1, 50, n_size) # Processing time
rul_vec = np.random.uniform(100, 150, n_size) # Degradation time
delta_vec = p_vec / rul_vec
job_df = pd.DataFrame(data={'job': range(n_size), 'p-time': p_vec, 'degradation': delta_vec})

# Algorithm Initialization (Genetic Algorithm.R)
PopSize = 200 # Population size
alpha_ga = 0.8 # Initialization percentage
CrossProb = 0.7 # Crossover probability
MutProb = 0.015 # Mutation probability
beta_ga = 0.2 # Replacement percentage
CycleGen = 20 # Restart cycle mechanism
epsilon_min = 0.2 # Low dispersion coefficient of variation
epsilon_max = 0.7 # High dispersion coefficient of variation
Rst = 0.25 # Restart mechanism percentage
MaxGen = 300 # Stopping criteria

### Functions ###

In [3]:
## Initialization of Population
def generate_pop_fun(p_init=False, p_popsize=PopSize):

    r_population = list()
    for p in range(int(p_popsize)):
        r_population.append(list()) # Generate n_size list as n_size is maximum amount of production batches
        r_population[p].append(list()) # Take care of Object Initialization in Python

        # In the initialization assign jobs either first-fit (prob = alpha_ga) or
        # sort descending by degradation and then first-fit (prob = 1 - alpha_ga)
        # In non-initialization cases, assign always random first-fit
        # alpha_ga*p_popsize candidates will be shuffled and then assigned first-fit
        if p <= alpha_ga*p_popsize or p_init == False:
            # Nils: Samples len(n_size) times in the interval of [0, len(n_size) - 1]. Equals R function sample(length(n_size)) with respect to the different indexes in Python
            v_jobs_ordered = sample(range(n_size), n_size)
        # (1-alpha_ga)*p_popsize candidates will be sorted by degradation ascending and then assigned first-fit
        else:
            # sort -> access column job -> get only the values -> transform numpy array into a list
            v_jobs_ordered = list(job_df.sort_values(by=["degradation"], ascending=False, inplace=False)["job"].values)

        # Assignment
        v_batch_i = 0
        for i in v_jobs_ordered:
            # Retrieve degradation to add to batch
            v_sum_batch_degradation = 0
            for h in r_population[p][v_batch_i]:
                v_sum_batch_degradation += job_df["degradation"].iloc[h]
            v_degradation_of_new_job = job_df["degradation"].iloc[i] # or delta_vec[i-1]
            # assign jobs to batch until batch reaches full deg
            # if current batch + new job >= delta (= full deg)
            if v_sum_batch_degradation + v_degradation_of_new_job >= delta:
                v_batch_i += 1
                r_population[p].append(list()) # Take care of object initialization

            r_population[p][v_batch_i].append(i)
            # r_population[[p]][sapply(r_population[[p]], is.null)] <- NULL # remove empty batches | Nils: Not needed anymore

    return r_population


## Calculate degradation for a batch ###
def degradation_fun(p_batch):

  # for each batch j calculate cumulative degradation
    v_cum_deg = 0
    # for each job k in batch j, retrieve degradation from delta_vec and cumulate
    for k in p_batch:
        v_cum_deg += delta_vec[k]
    return v_cum_deg


## Calculate cost function
def ga_cost_fun(p_candidate, p_con_offset = 0, p_pow_offset = 1):

    r_cost_ga = 0
    for jj in range(len(p_candidate)-1): # for each batch (last batch does not incur costs)
        #print(jj)
        v_deg_ga = theta_deg + degradation_fun(p_candidate[jj])
        #print(v_deg_ga)
        #print(degradation_fun(p_candidate[jj]))
        # set degradation to initial degradation +
        # sum of degradation of all jobs in batch jj
        r_cost_ga += cost_zero + ( cost_f - cost_zero ) * v_deg_ga

    # Return cost. If sd parameters are set, substract constant offset (p_con_offset)
    # and raise to power of power offset (p_pow_offset)
    return (r_cost_ga-p_con_offset)**p_pow_offset


## Calculate fitness function
def fitness_fun(p_candidate):

    ## Calculate affinity/fitness = reciprocal of cost
    r_affinity = 1/ga_cost_fun(p_candidate)
    return r_affinity


## Tournament selection
# Choose the fitter of two candidates for entire population
def tournament_fun(p_population):

    # Shuffle order of candidates
    # Nils: Samples len(p_population) times in the interval of [0, len(p_population) - 1]. Equals R function sample(length(p_population)) with respect to the different indexes in Python
    tournament_order = sample(range(len(p_population)), len(p_population))
    # Create new list for tournament "winners"
    r_population_new = list()
    v_index_halver = int(len(p_population)/2) # Floor
    for i in range(v_index_halver):
        # Check which candidate score a higher fitness value
        if fitness_fun(p_population[tournament_order[i]]) > fitness_fun(p_population[tournament_order[v_index_halver + i]]):
            # First candidate is fitter, add to new population
            r_population_new.append(p_population[tournament_order[i]])
        else:
            # Second candidate is fitter, add to new population
            r_population_new.append(p_population[tournament_order[v_index_halver + i]])

    return r_population_new


def fill_child_fun(p_parent, p_child):

    # Loop through batches j of parent
    for j in range(len(p_parent)):
        # Loop through jobs k of parent batch j
        for k in range(len(p_parent[j])):
            v_job_contained = False
            # Loop through batches l of child
            for l in range(len(p_child)):
                # Check if job k of batch j is contained in batch l of child i
                # If yes, set flag to true
                if p_parent[j][k] in p_child[l]:
                    v_job_contained = True
            # If job k is not contained in any batch of child i, insert it into child first-fit
            if not v_job_contained:
                # Loop through batches l of child and see if there is still space for job k of parent batch j
                for l in range(len(p_child)):
                    v_job_inserted = False
                    if degradation_fun(p_child[l]) + delta_vec[p_parent[j][k]] < delta:
                        v_job_inserted = True
                        p_child[l].append(p_parent[j][k])
                        break

                # If all batches are full, open up new batch
                if not v_job_inserted:
                    p_child.append(list())
                    p_child[-1].append(p_parent[j][k])

    return p_child


## Crossover of two parents
# p_population must have an even length() as it gets halved!
# Optional Open Issue (ToDo, low prio): When uneven, residual parent crosses over with random other parent
def crossover_fun(p_population):

    r_children1 = list()
    r_children2 = list()

    if len(p_population)/2 % 2 != 0:
        print("Length of p_population must be even")

    # Nils: Samples len(p_population) times in the interval of [0, len(p_population) - 1]. Equals R function sample(length(p_population)) with respect to the different indexes in Python
    crossover_order = sample(range(len(p_population)), len(p_population))
    for i in range(int(len(p_population)/2)):

        # Manage object initialization
        r_children1.append(list())
        r_children2.append(list())

    # Check if CrossProb "fires" and children are created
        if CrossProb >= np.random.uniform(0, 1):

            # Assign parents
            v_parent_1 = list()
            v_parent_2 = list()
            for batches1 in p_population[crossover_order[i]]:
                v_parent_1.append(batches1)
            for batches2 in p_population[crossover_order[i + int(len(p_population)/2)]]:
                v_parent_2.append(batches2)

            v_parents = v_parent_1 + v_parent_2

            # 1. In the first phase, blocks from both parents are sorted in
            # the order of non-increasing degradation

            # Create df with batch ids and cumulative degradation (initialized with 0s)
            v_parents_id_df = pd.DataFrame(data={'batch': range(len(v_parents)), 'deg': [0] * len(v_parents)})
            # for each batch j calculate cumulative degradation
            for j in range(len(v_parents_id_df)):
                # for each job k in batch j, retrieve degradation from delta_vec and cumulate
                v_parents_id_df["deg"].iloc[j] = degradation_fun(v_parents[j])

            # order batch index df in descending order of cumulative degradation
            v_parents_id_df = v_parents_id_df.sort_values(by=['deg'], ascending=False)

            # 2. Next, starting from two empty offspring, we copy the
            # fullest non-overlapping blocks from parents. In other
            # words, a block is copied in both offspring only if it contains
            # no duplicated job.
            # Always add first batch
            r_children1[i].append(v_parents[v_parents_id_df["batch"].iloc[0]])
            r_children2[i].append(v_parents[v_parents_id_df["batch"].iloc[0]])
            # TODO : Work with counter
            for j in range(1, len(v_parents_id_df)):
                # Check if any job of batch j is in any batch < j-1

                # set marker that any job of current v_parents[[v_parents_id_df$batch[[j]]]]
                # is already containend in children to FALSE
                v_job_contained = False
                # Loop over all batches of children
                for k in range(len(r_children1[i])):
                    # Check if any job in parent is in any children
                    # (only one children must be checked, as they are identical)
                    if any(x in v_parents[v_parents_id_df["batch"].iloc[j]] for x in r_children1[i][k]):
                        v_job_contained = True

                # If no job of parent batch j was found in any children,
                # append batch j to both children
                if not v_job_contained:
                    r_children1[i].append(v_parents[v_parents_id_df["batch"].iloc[j]])
                    r_children2[i].append(v_parents[v_parents_id_df["batch"].iloc[j]])

            ### Until now, both children r_children[[i]] and r_children[[i*2]] are completely identical
            ### They might be missing single jobs that could not be assigned, because they were in a
            ### a parent batch that also contained jobs that were already assigned to any child
            ### Now in step 3, we scan v_parent_1 from left to right and assign missing jobs to r_children[[i]]
            ### These jobs are inserted first fit, that means, that they can be assigned to batch that are not fully degraded
            ### If this is not possible, we, of course, open up a new batch
            ### For r_children[[i*2]], we do the same with v_parent_2
            ### Now, the children are different as the first inherits more from the first parent and vice versa
            # Loop through batches j of parent
            r_children1[i] = fill_child_fun(v_parent_1, r_children1[i])
            r_children2[i] = fill_child_fun(v_parent_2, r_children2[i])

    r_children = r_children1 + r_children2

    # Delete empty batches via List Comprehension which were skipped by CrossProb
    r_children = [x for x in r_children if x] # if list returns true if not empty

    return r_children


def mutation_fun(p_population, p_mutprob):

    for i in range(len(p_population)):
        # After the offspring are generated from the selection and crossover,
        # the offspring chromosomes may be mutated. Like crossover, there is
        # a mutation probability. If a randomly selected floating-point value
        # is less than the mutation probability (p_mutprob), mutation is performed
        # on the offspring; otherwise, no mutation occurs.
        if p_mutprob >= np.random.uniform(0, 1):
            # For each mutation operation, the number of permutations is randomly
            # chosen between (5%*n + 1) and (15%*n + 1) where n is the number jobs
            # Nils: sample() returns a list in python. Therefore, the [0] at the end is used
            for j in range(sample(range(math.ceil(0.05 * n_size + 1), math.floor(0.15 * n_size + 1)), 1)[0]):
                # Swapping, if possible, two randomly selected jobs from two different
                # blocks. We only allow mutations that guarantee the feasibility of
                # the obtained solutions. Thus, the maximal threshold Î (delta) must be
                # respected for each block. We repeat until a valid swap is found.
                # Create a batch & job index data.frame and shuffle it
                v_batch_job_df = pd.DataFrame(columns=["batch", "job"])
                for k in range(len(p_population[i])):
                    # Nils: concat is the Python equivalent for rbind() and .tolist() converts the numpy dtypes into native dtypes (numpy.int32 -> int)
                    v_batch_job_df = pd.concat([v_batch_job_df, pd.DataFrame({'batch': np.repeat(np.array([k]), len(p_population[i][k]), axis=0).tolist(), 'job': list(range(len(p_population[i][k])))})], ignore_index=True)

                # Nils: Samples len(v_batch_job_df) times in the interval of [0, len(v_batch_job_df) - 1]. Equals R function sample(nrow(v_batch_job_df)) with respect to the different indexes in Python
                v_batch_job_df = v_batch_job_df.loc[sample(range(len(v_batch_job_df)), len(v_batch_job_df))]
                v_batch_job_df = v_batch_job_df.reset_index(drop=True)
                # Loop over random data.frame in case the first result has no valid swaps
                for k in range(len(v_batch_job_df)): # Nils: We have to loop over the edited indexes since df.loc selects the data based on the index and not the actual stored dataframe
                    v_stop = False
                    # Sample batch/job combination
                    v_batch_job1 = list(v_batch_job_df.loc[k])
                    # Filter all other batches
                    # Nils: Returns values which do not equal v_batch_job1[0] (=v_batch_job1$batch) and sorts the results based on the index. Equals the dplyr %>% + filter function
                    v_batch2_job_df = v_batch_job_df[v_batch_job_df['batch'] != v_batch_job1[0]]
                    v_batch2_job_df = v_batch2_job_df.reset_index(drop=True)
                    # Sample batch/job combination from another batch
                    for l in range(len(v_batch2_job_df)):
                        v_batch_job2 = list(v_batch2_job_df.loc[l])
                        # Try swap
                        v_temp_population = p_population[i]
                        v_job1 = v_temp_population[v_batch_job1[0]][v_batch_job1[1]]
                        v_job2 = v_temp_population[v_batch_job2[0]][v_batch_job2[1]]
                        v_temp_population[v_batch_job1[0]][v_batch_job1[1]] = v_job2
                        v_temp_population[v_batch_job2[0]][v_batch_job2[1]] = v_job1
                        # Check if swap lead to two feasible batches
                        if degradation_fun(v_temp_population[v_batch_job1[0]]) < delta and degradation_fun(v_temp_population[v_batch_job2[0]]) < delta:
                            p_population[i] = v_temp_population
                            # We do not need to test other feasible solutions
                            v_stop = True
                            break

                    if v_stop:
                        break # Break the outer loop when the flag is fired
                    # If all batches have been run through and break has not fired yet,
                    # No swap is feasible for p_population[[i]]
                    if (k+1) == len(v_batch_job_df):
                        print("No feasible swap found for chromosome %i" % i) # TODO add generation index to print

    return p_population


def CoV_fun(p_population):

    # Mean cost of the whole population
    v_costs = list()
    for i in p_population:
        v_costs.append(ga_cost_fun(i))
    v_mean = mean(v_costs)

    # Standard deviation of the whole population
    v_costs = list()
    for i in p_population:
        v_costs.append(ga_cost_fun(i, p_con_offset=v_mean, p_pow_offset=2))
    v_sd = math.sqrt(mean(v_costs))
    r_cov = v_sd/v_mean
    return r_cov


def exploration_fun(p_population):

    v_new_rand = generate_pop_fun(p_init=False, p_popsize = Rst*PopSize)
    v_fitness_values = list()
    for i in p_population:
        v_fitness_values.append(fitness_fun(i))
    v_pop_fitness = pd.DataFrame(v_fitness_values, columns = ['Fitness Values'])
    v_pop_fitness = v_pop_fitness.sort_values(by=['Fitness Values'], ascending=False)
    v_pop_order = list(v_pop_fitness.index.values)

    v_fittest_Rst = list()
    for i in range(len(p_population) - round(Rst*PopSize)):
        v_fittest_Rst.append(p_population[v_pop_order[i]])

    r_population = v_fittest_Rst + v_new_rand
    return r_population


## Mutate fittest Rst*PopSize individuals and append to p_population
def exploitation_fun(p_population):

    v_fitness_values = list()
    for i in p_population:
        v_fitness_values.append(fitness_fun(i))
    v_pop_fitness = pd.DataFrame(v_fitness_values, columns = ['Fitness Values'])
    v_pop_fitness = v_pop_fitness.sort_values(by=['Fitness Values'], ascending=False)
    v_pop_order = list(v_pop_fitness.index.values)

    v_fittest_Rst = list()
    for i in range(round(Rst*PopSize)):
        v_fittest_Rst.append(p_population[v_pop_order[i]])

    r_population = p_population + mutation_fun(v_fittest_Rst, 1)
    return r_population


## Construct New Population with PopSize from parents and children
def replacement_fun(p_population):

    v_fitness_values = list()
    for i in p_population:
        v_fitness_values.append(fitness_fun(i))
    v_pop_fitness = pd.DataFrame(v_fitness_values, columns = ['Fitness Values'])
    v_pop_fitness = v_pop_fitness.sort_values(by=['Fitness Values'], ascending=False)
    v_pop_order = list(v_pop_fitness.index.values)
    # Initialize new population list
    r_new_population = list()
    v_cutoff = round(beta_ga * len(p_population))

    # Insert worst beta_ga share of individuals
    for i in range(v_cutoff):
        r_new_population.append(p_population[v_pop_order[len(p_population) - v_cutoff + i]])

    # Fill up with best individuals
    v_counter = 0
    for i in range(v_cutoff, PopSize):
        r_new_population.append(p_population[v_pop_order[v_counter]])
        v_counter += 1

    return r_new_population

### Start of Programm ###

In [4]:
# Initialization
population = generate_pop_fun(p_init=True, p_popsize=PopSize)
fittest_candidate = list()
v_fittest = 0.0
for i in range(MaxGen):
    print("Generation: %i" % (i+1))
    # Selection
    parents = tournament_fun(population)
    # Crossover
    children = crossover_fun(parents)
    # Mutation
    mut_children = mutation_fun(children, MutProb)
    total_population = list()
    total_population = population.copy() + mut_children
    if i % CycleGen == 0:
        # If coefficient of variation is higher than epsilon_max, delete worst Rst*PopSize
        # individuals and generate random new ones (receptor editing)
        if CoV_fun(total_population) < epsilon_min:
            print("CoV %f is smaller than epsilon_min (%f). Explore" % (CoV_fun(total_population), epsilon_min))
            new_population = exploration_fun(total_population)
            #new_population = replacement_fun(new_population)
            # If coefficient of variation is higher than epsilon_max, generate Rst*PopSize
            # individuals by mutating best solutions and injecting them into population
        elif CoV_fun(total_population) > epsilon_max:
            print("CoV %f is higher than epsilon_max (%f). Exploit." % (CoV_fun(total_population), epsilon_min))
            new_population = exploitation_fun(total_population)
            #new_population = replacement_fun(new_population)
        else:
            new_population = total_population
            # If CoV is moderate, just replace
            # new_population = replacement_fun(total_population)
    else:
        new_population = replacement_fun(total_population)

    # Store fittest candidate in variable
    v_fitness_values = list()
    for i in new_population:
        v_fitness_values.append(fitness_fun(i))
    v_pop_fitness = pd.DataFrame(v_fitness_values, columns = ['Fitness Values'])
    v_pop_fitness = v_pop_fitness.sort_values(by=['Fitness Values'], ascending=False)
    pop_fitness = list(v_pop_fitness.index.values)
    if not fittest_candidate or fitness_fun(new_population[pop_fitness[0]]) > v_fittest:
        fittest_candidate = new_population[pop_fitness[0]].copy()
        v_fittest = fitness_fun(fittest_candidate)
        print("A fittest candidate was found with a fitness of %f" % fitness_fun(fittest_candidate))
    else:
        print("No fitter candidate was found")

Generation: 1
A fittest candidate was found with a fitness of 0.000220
Generation: 2
A fittest candidate was found with a fitness of 0.000233
Generation: 3
A fittest candidate was found with a fitness of 0.000252
Generation: 4
No fitter candidate was found
Generation: 5
A fittest candidate was found with a fitness of 0.030696
Generation: 6
No fitter candidate was found
Generation: 7
No fitter candidate was found
Generation: 8
No fitter candidate was found
Generation: 9
No fitter candidate was found
Generation: 10
No fitter candidate was found
Generation: 11
No fitter candidate was found
Generation: 12
No fitter candidate was found
Generation: 13
No fitter candidate was found
Generation: 14
No fitter candidate was found
Generation: 15
No fitter candidate was found
Generation: 16
No fitter candidate was found
Generation: 17
No fitter candidate was found
Generation: 18
No fitter candidate was found
Generation: 19
No fitter candidate was found
Generation: 20
No fitter candidate was found
G

### Battery Simulation

In [8]:
# Init packages
from prog_models.models import BatteryCircuit
import numpy as np
import pandas as pd
import math
import sys

# Init machine model
batt = BatteryCircuit() # Create physical model

def get_rul(p_batch):
    p_batch.time = p_batch.time.copy().cumsum()
    #print(p_batch); sys.stdout.flush()
    # Loading function
    # t = time
    # loading_df = two column df with first column containing the cumulated production time and
    # second column containing the loading / intensity
    def future_loading(t, x=None):
        r_loading = p_batch[p_batch.time >= t].iloc[0].intensity.copy()
        return {'i': r_loading}
    options = { #configuration for this sim
        'save_freq': 10,  # Frequency at which results are saved (s)
        'horizon': math.floor(p_batch.time.iloc[-1])  # Maximum time to simulate (s) - This is a cutoff. The simulation will end at this time, or when a threshold has been met, whichever is first
    }
    (times, inputs, states, outputs, event_states) = batt.simulate_to_threshold(future_loading, **options)
    return (event_states[-1]["EOD"])
    #return (times, inputs, states, outputs, event_states)

In [14]:
batch  = pd.DataFrame({'time': [400, 300, 200, 100], 'intensity': [2, 3, 4, 5]})
get_rul(batch)

0.6149080836706061